# Spike Detection GUI Demo

Interactive walkthrough of all 4 spikedetect GUIs using real recording data.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import os
import spikedetect as sd
from spikedetect.gui import FilterGUI, TemplateSelectionGUI, ThresholdGUI, SpotCheckGUI
from spikedetect.pipeline.filtering import SignalFilter
from spikedetect.pipeline.peaks import PeakFinder
from spikedetect.pipeline.template import TemplateMatcher
print('Ready!')

## Load recording

In [ ]:
import os
mat_file = os.path.expanduser('~/Documents/GitHub/spikeDetection/LEDFlashTriggerPiezoControl_Raw_240430_F1_C1_5.mat')
rec = sd.load_recording(mat_file)
fs = rec.sample_rate
print(f'{rec.name}: {rec.n_samples} samples, {fs} Hz, {rec.duration:.1f} s')

# Use saved params if available, otherwise defaults
if rec.result is not None and rec.result.params.spike_template is not None:
    params = rec.result.params
    print('Using saved detection parameters.')
else:
    params = sd.SpikeDetectionParams.default(fs=fs)
    print('Using default parameters.')

## Step 1: FilterGUI
Adjust bandpass filter sliders. When you're happy, press **Enter** on the figure to continue.

**Note:** The interactive figure appears below. Click on it first to give it focus before pressing keys.

In [ ]:
# Build the FilterGUI but display it manually for the notebook
filter_gui = FilterGUI(rec.voltage, params)
filter_gui._apply_filter()
filter_gui._build_figure()
filter_gui._update_plots()
plt.show()

In [ ]:
# Run this cell when you're done adjusting sliders to save your settings
params = filter_gui.params
print(f'hp={params.hp_cutoff} Hz, lp={params.lp_cutoff} Hz, diff={params.diff_order}, pol={params.polarity:+d}')
plt.close(filter_gui.fig)

## Step 2: Filter data & select template
Click on 3-5 clear spike peaks, then press **Enter**.

In [ ]:
start_point = round(0.01 * fs)
unfiltered_data = rec.voltage[start_point:]
filtered_data = SignalFilter.filter_data(
    unfiltered_data, fs=fs,
    hp_cutoff=params.hp_cutoff, lp_cutoff=params.lp_cutoff,
    diff_order=params.diff_order, polarity=params.polarity,
)

template_gui = TemplateSelectionGUI(filtered_data, params)
template = template_gui.run()

if template is not None:
    params.spike_template = template
    print(f'Template length: {len(template)} samples')
else:
    print('Keeping existing template.')

## Step 3: Detect spikes & adjust thresholds
Click scatter plot to move thresholds. Press **b** to toggle between distance/amplitude. Press **Enter** to accept.

In [ ]:
# Initial detection
result = sd.detect_spikes(rec, params)
print(f'Initial detection: {result.n_spikes} spikes')

# Threshold GUI
spike_locs = PeakFinder.find_spike_locations(
    filtered_data, peak_threshold=params.peak_threshold,
    fs=fs, spike_template_width=params.spike_template_width,
)
match_result = TemplateMatcher.match(
    spike_locs=spike_locs, spike_template=params.spike_template,
    filtered_data=filtered_data, unfiltered_data=unfiltered_data,
    spike_template_width=params.spike_template_width, fs=fs,
)
threshold_gui = ThresholdGUI(match_result, params)
params = threshold_gui.run()
print(f'distance_threshold={params.distance_threshold:.2f}, amplitude_threshold={params.amplitude_threshold:.3f}')

## Step 4: Re-detect & spot-check
Review spikes: **y**=accept, **n**=reject, arrow keys to adjust, click raster/scatter to navigate. Press **Enter** when done.

In [ ]:
result = sd.detect_spikes(rec, params)
print(f'Final detection: {result.n_spikes} spikes')

spotcheck = SpotCheckGUI(rec, result)
result = spotcheck.run()
print(result.summary())